In [1]:
# Denpendances
import numpy as np
import pandas as pd
from dLoader import DataLoader

In [2]:
# Generating gain and loss percentage data
def clip_dataframe(df, p):
    # Batching for test data
    batches = len(df) // p
    max_length = batches * p
    return df.iloc[-max_length:].copy()

def simple_gain_loss(data, period):
    # Get gain and loss values base on start of period and
    # end of period prices
    ndf = clip_dataframe(data, period)
    # Base period prices
    base = np.array(ndf['Close']).reshape(-1, period)[:, 0]
    # Shifting one day to avoid the end period price is the
    # base price price
    shifted = ndf.shift(-1).fillna(method='ffill')
    target = np.array(shifted['Close']).reshape(-1, period)
    # Calculate gain and loss price array
    gain = target.max(1) / base - 1
    loss = target.min(1) / base - 1
    return gain.mean(), loss.mean()

def average_daily_fluctuation(data, minute=30):
    # Calculate an average daily fluctuation percentage value
    hl = data['High'] / data['Low'] - 1
    minutes = 7.5 * (60 / minute)
    return hl.mean() / minutes

def get_values(data, period, minute=30):
    # Getting values of sell limit, stop loss and fluctuation percentage
    sell_limit, stop_loss = simple_gain_loss(data, period)
    fluct = average_daily_fluctuation(data, minute)
    return sell_limit, stop_loss, fluct

In [44]:
# Static back test
def not_so_simple_gain_loss(data, period):
    ndf = clip_dataframe(data, period)
    # Base
    base = np.array(ndf['Close']).reshape(-1, period)[:, 0]
    # Expanding dimension
    base = np.expand_dims(base, 1)
    ndf = clip_dataframe(data, period)
    # base
    base = np.array(ndf['Close']).reshape(-1, period)[:, 0]
    base = np.expand_dims(base, 1)
    # shifted
    shift = ndf.shift(-1).fillna(method='ffill')
    # High
    high = np.array(shift['High']).reshape(-1, period)
    # Low
    low = np.array(shift['Low']).reshape(-1, period)
    # Open
    Open = np.array(shift['Open']).reshape(-1, period)
    # Close
    close = np.array(shift['Close']).reshape(-1, period)
    # Gaining percentage
    high_gains = high / base - 1
    low_gains = low / base - 1
    close_gains = close / base - 1
    open_gains = Open / base - 1
    return open_gains, high_gains, low_gains, close_gains, base.reshape(-1)

def calculate_gains(data, period, sell_limit, stop_loss, fluct):
    open_gains, high_gains, low_gains, close_gains, base = not_so_simple_gain_loss(data, period)
    # Getting gained percentage 
    length = len(open_gains)
    arr = []
    for i in range(length):
        sold = False
        for p in range(period):
            # Main stategy
            if open_gains[i, p] < stop_loss:
                arr.append(open_gains[i, p])
                sold = True
                break
            elif high_gains[i, p] > sell_limit:
                arr.append(high_gains[i, p] - fluct)
                sold = True
                break
            elif low_gains[i, p] < stop_loss:
                arr.append(stop_loss)
                sold = True
                break
        if not sold:
            arr.append(close_gains[i, -1])
    return base, np.array(arr)

def calculate_capital_gain(base, gains, capital=1000, max_share=1000):
    if base[0] > capital:
        capital = base[0] * 10
    original_capital = capital
    length = len(base)
    for i in range(length):
        gain = base[i] * gains[i]
        share = capital // base[i]
        if share > max_share:
            share = max_share
        capital += gain * share
    return capital / original_capital - 1

In [72]:
# Get stock symbol in Database
import os
folder = os.path.abspath('Database')
files = os.listdir(folder)
symbols = [f.split('.')[0] for f in files]
symbols = sorted(symbols)
symbols

['AAPL', 'AMD', 'FCEL', 'MSFT', 'NVDA', 'TSLA']

In [74]:
for symbol in symbols:
    d1 = DataLoader(symbol).get_data('2018-01-01', '2020-12-31')
    d2 = DataLoader(symbol).get_data('2021-01-01', '2021-12-31')
    print('Checking {}'.format(symbol))
    print(' Period | Sell Limit % |  Stop Loss % | Capital Gain % ')
    cg = []
    for i in range(3, 21):
        sell_limit, stop_loss, fluct = get_values(d1, i)
        base, gains = calculate_gains(d2, i, sell_limit, stop_loss, fluct)
        capital_gain_percentage = calculate_capital_gain(base, gains)
        print(' {:>6d} | {:>12.2f} | {:>12.2f} | {:>14.2f} '.format(
            i, sell_limit * 100, stop_loss * 100,
            capital_gain_percentage * 100))
        cg.append(capital_gain_percentage)

    print('\nBase on static 3 - 20 days buy sell pattern: ')
    cg = np.array(cg)
    risk = len(cg[cg < 0]) / len(cg)
    leverage = abs(cg.max() / cg.mean())
    print('Risk: {:>6.2f}% | Leverage: {:>6.2f}'.format(
        risk * 100, leverage))
    print('\n')

Checking AAPL
 Period | Sell Limit % |  Stop Loss % | Capital Gain % 
      3 |         1.52 |        -0.97 |          46.84 
      4 |         2.03 |        -1.20 |          25.99 
      5 |         2.62 |        -1.54 |          19.67 
      6 |         2.82 |        -1.92 |          16.20 
      7 |         3.65 |        -1.77 |          15.85 
      8 |         3.91 |        -1.88 |          25.53 
      9 |         4.35 |        -2.25 |          -3.25 
     10 |         4.68 |        -2.43 |          19.79 
     11 |         4.56 |        -2.92 |          21.01 
     12 |         4.67 |        -3.19 |          17.27 
     13 |         5.18 |        -3.43 |          24.08 
     14 |         6.16 |        -3.10 |          17.08 
     15 |         5.75 |        -4.17 |          22.24 
     16 |         6.53 |        -3.13 |           6.58 
     17 |         6.98 |        -3.84 |          36.45 
     18 |         7.17 |        -3.82 |           3.90 
     19 |         6.54 |        -4